In [1]:
import Base: iterate,max, exp, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero
import Test: @test, @testset

In [2]:
2^2

4

# Define dual number

In [3]:
struct Dual{T <: Number} <: Number
    x::T
    dx::T
end
Dual(n::Integer, d::Float64) = Dual(promote(n, d)...)
Dual(n::Float64, d::Integer) = Dual(promote(n, d)...)

Dual

In [4]:
function convert(::Type{Dual{T}}, x::T) where {T}
   Dual(x, zero(x)) 
end
function convert(::Type{Dual{T}}, x::Dual{S}) where {S, T}
    Dual(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Dual{T}}, x::T) where {T <: Number}
    Dual(x, zero(x))
end
function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Dual(x_as_T, zero(x_as_T))
end

convert (generic function with 187 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(x.x), zero(x.dx))
end

zero (generic function with 23 methods)

In [6]:
function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S,T)}
end

promote_rule (generic function with 126 methods)

In [7]:
function extract_derivative(xdx::Dual)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

# Differential operator

In [8]:
function derivativeСalculation(f, value)
    function df(x)
        xdx = Dual(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df(value)
end
function derivativeСalculation(f,i::Integer,values...)
    print("DUPA")
    function df(valuesS)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(valuesS)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df(values)
end
function derivativeСalculation(f,i::Integer)
    function df(xs...)
        xarr = [(j != i ? x : Dual(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df
end

derivativeСalculation (generic function with 3 methods)

In [9]:
function +(x::Dual, y::Dual)
    Dual(x.x+y.x, x.dx+y.dx)
end
function -(x::Dual, y::Dual)
    Dual(x.x-y.x, x.dx-y.dx)
end
function -(x::Dual)
    Dual(-x.x, -x.dx)
end
function *(x::Dual, y::Dual)
    Dual(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Dual, y::Dual)
    Dual(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end
function /(x::Array, y::Number)
    return [i / y for i in x]
end
function ^(a::Dual, x::Dual)
    Dual(a.x^x.x, a.x^x.x*log(a.x)*x.dx)
end

^ (generic function with 67 methods)

In [10]:
function sqrt(x::Dual)
    Dual(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 21 methods)

In [11]:
function exp(x::Dual)
    return Dual(exp(x.x), exp(x.x)*x.dx)
end
function exp(xs::Array)
    return [exp(x) for x in xs]
end
function sin(x::Dual)
    return Dual(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Dual)
    return Dual(cos(x.x), -sin(x.x)*x.dx)
end

function tan(x::Dual)
    return Dual(tan(x.x),(1/cos(x.x)^2)*x.dx)
end

tan (generic function with 13 methods)

In [12]:
function max(x::Dual)
    return Dual(max(0,x.x), x.x < 0 ? 0 : 1 * x.dx)
end

max (generic function with 13 methods)

In [13]:
function iterate(iter::Dual, state=1)
    if state > length(iter.val)
        return nothing
    end
    return (iter[state],state+1)
end

iterate (generic function with 212 methods)

In [14]:
@test begin
    x = randn()
    isapprox(D(exp)(x), exp(x))
end

Error During Test at In[14]:1
  Test threw exception
  Expression: begin
    x = randn()
    isapprox((D(exp))(x), exp(x))
end
  UndefVarError: D not defined
  Stacktrace:
   [1] top-level scope at In[14]:1
   [2] include_string(::Function, ::Module, ::String, ::String) at ./loading.jl:1091
   [3] softscope_include_string(::Module, ::String, ::String) at /home/ligerd/.julia/packages/SoftGlobalScope/u4UzH/src/SoftGlobalScope.jl:65
   [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/ligerd/.julia/packages/IJulia/e8kqU/src/execute_request.jl:67
   [5] #invokelatest#1 at ./essentials.jl:710 [inlined]
   [6] invokelatest at ./essentials.jl:709 [inlined]
   [7] eventloop(::ZMQ.Socket) at /home/ligerd/.julia/packages/IJulia/e8kqU/src/eventloop.jl:8
   [8] (::IJulia.var"#15#18")() at ./task.jl:356
  


LoadError: [91mThere was an error during testing[39m

In [15]:
function f(x)
    return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
end
fprime = D(f,3)

LoadError: [91mUndefVarError: D not defined[39m

In [16]:
fprime(3)

LoadError: [91mUndefVarError: fprime not defined[39m

In [16]:
function f(x)
#     return exp(-x/pi)*sin(2.0*pi*sqrt(2)*x)
    return sin(x*x)
end
fprime = D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [17]:
D(2)

(::var"#df#1"{Int64}) (generic function with 1 method)

In [15]:
function linear(x, m, b)
    return m*x+b
end


linear (generic function with 1 method)

In [18]:
derivativeСalculation(linear,1,1.2,2,3)

DUPA

2.0

In [33]:
dldx(1.2,2,3)

1.0

In [25]:
function relu(x)
    return max(0,x)
end

relu (generic function with 1 method)

In [26]:
function f(x)
    return x+relu(x*x)
end

f (generic function with 1 method)

In [27]:
d= D(f)

(::var"#df#1"{typeof(f)}) (generic function with 1 method)

In [28]:
d(-2)

LoadError: [91mMethodError: no method matching isless(::Infinitesimal{Int64}, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  isless([91m::Missing[39m, ::Any) at missing.jl:87[39m
[91m[0m  isless([91m::AbstractFloat[39m, ::Real) at operators.jl:167[39m
[91m[0m  isless([91m::Real[39m, ::Real) at operators.jl:355[39m
[91m[0m  ...[39m

In [29]:
function softmax(vector::Array)
    e = exp(vector)
    return e / sum(e)
end

softmax (generic function with 1 method)

In [30]:
d=D(softmax)

(::var"#df#1"{typeof(softmax)}) (generic function with 1 method)

In [31]:
d(2, [1,2,3])

LoadError: [91mMethodError: no method matching (::var"#df#1"{typeof(softmax)})(::Int64, ::Array{Int64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  df(::Any) at In[11]:2[39m

In [14]:
function ftest(x)
    return 3^x
end

ftest (generic function with 1 method)

In [15]:
d=D(ftest)

(::var"#df#1"{typeof(ftest)}) (generic function with 1 method)

In [16]:
d(2.0)

Dual{Float64}(2.0, 1.0)

9.887510598012987

In [44]:
Dual(2,4)+Dual(4.4, 2.1)

Dual{Float64}(6.4, 6.1)

In [47]:
Dual(2,4.3)

Dual{Float64}(2.0, 4.3)

In [41]:
function tgTest(x)
    return tan(x)
end

tgTest (generic function with 1 method)

In [42]:
d=D(tgTest)

(::var"#df#1"{typeof(tgTest)}) (generic function with 1 method)

In [43]:
d(5)

Dual{Int64}(5, 1)

12.427881707458354

In [40]:
tan(2)

-2.185039863261519

In [44]:
sin(2)

0.9092974268256817